In [108]:
import numpy as np
import os
import re
import csv

percentile_var=0.7 #for variance selection
percentile_uni=0.8 #for univariate selection

feature_folder= './FeatureSampleFoodClassification_100/edge_array_sobel'
output_folder_var= feature_folder + '_var' + str(percentile_var) +  '/'
output_folder_uni= feature_folder + '_uni' + str(percentile_uni) + '/'
output_folder_tree= feature_folder + '_tree/'

feature_folder=feature_folder + '/'

if not os.path.exists(output_folder_var):
    os.makedirs(output_folder_var)
    
if not os.path.exists(output_folder_uni):
    os.makedirs(output_folder_uni)

if not os.path.exists(output_folder_tree):
    os.makedirs(output_folder_tree)
    
files = [f for f in os.listdir(feature_folder) if os.path.isfile(os.path.join(feature_folder, f))]
features = np.array([np.load(os.path.join(feature_folder, f)) for f in files])
simplifiedNames = [re.sub(r'\.npy*$','',s) for s in files]

n=len(simplifiedNames)

names_features=sorted(zip(simplifiedNames, features), key= lambda x : x[0])
features=[x[1] for x in names_features]

# Load training info
with open('./sample_food_no_food.csv') as f:
    food_no_food = [{k: v for k, v in row.items()}
        for row in csv.DictReader(f, skipinitialspace=True)]

labels = np.array([x[1] for x in sorted([(k['id'], np.int64(k['is_food'])) for k in food_no_food], key= lambda x : x[0])])

Low Variance removal (no effect for WS), just for binary features!

In [109]:
from sklearn.feature_selection import VarianceThreshold

sel = VarianceThreshold(threshold=(percentile_var * (1 - percentile_var)))
selection_var=sel.fit_transform(features)

for i in range(0,n):
    np.save(output_folder_var + simplifiedNames[i],selection_var[i])


Univariate feature selection 

In [110]:
from sklearn.feature_selection import SelectPercentile
from sklearn.feature_selection import chi2

selection_uni= SelectPercentile(chi2, percentile_uni*100).fit_transform(features,labels)

for i in range(0,n):
    np.save(output_folder_uni + simplifiedNames[i],selection_uni[i])


Tree-based feature selection

In [111]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel

clf = ExtraTreesClassifier()
clf = clf.fit(features, labels)
model = SelectFromModel(clf, prefit=True)
selection_tree = model.transform(features)

for i in range(0,n):
    np.save(output_folder_tree + simplifiedNames[i],selection_tree[i])